# Usar um pipeline para ler e processar um conjunto de ficheiros e usar esse mesmo pipeline como 'conjunto de treino' #

In [1]:
import glob
import numpy as np
import tensorflow as tf

2022-05-06 14:16:21.106765: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


A api, tf.data.Dataset , permite criar um pipeline de leitura de ficheiros e seu eventual processamento, usando a função 'map':
https://www.tensorflow.org/guide/data
,
https://www.tensorflow.org/guide/data_performance
.

Podemos fornecer diretamente um tfdata.Dataset ao método fit é uma forma muito eficiente de efetuar o treino de um modelo keras: https://www.tensorflow.org/guide/keras/train_and_evaluate#training_evaluation_from_tfdata_datasets


No que se segue exemplificaremos a utilização desta api num conjunto de ficheiros do 2ª trabalho. O tf.data.Dataset permite criar 'batches' que podem sevir de inputs e labels a um modelo keras que faça a segmentação dos sons cardíacos.

In [2]:
dir = './'
list_npy_files = glob.glob(dir + '*.npy')
fnames_dataset = tf.data.Dataset.from_tensor_slices(list_npy_files)

2022-05-06 14:16:22.041032: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-06 14:16:22.090428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-06 14:16:22.090632: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1660 computeCapability: 7.5
coreClock: 1.815GHz coreCount: 22 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 178.86GiB/s
2022-05-06 14:16:22.090648: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-06 14:16:22.092238: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-06 14:16:22.092267: I tensorflow/stream_executor/platform/d

Podemos aceder um a um aos elementos (ou batches de elementos) de um dataset

In [3]:
it = iter(fnames_dataset)
ex = next(it)
ex

<tf.Tensor: shape=(), dtype=string, numpy=b'./2530_MV_sigTarg.npy'>

A partir deste dataset inicial, fnames_dataset, vamos criar um cujo conteudo é o resultado de ler cada um dos ficheiros npy.

Começamos por uma função auxiliar que vai ser usada pela função 'map' para ler o conteúdo de cada um dos ficheiros do fnames_dataset e criar um novo dataset com estes conteúdos.

In [4]:
def read_npy(fname):
    """fname should be a npy file"""
    fname = fname.decode()
    recData = np.load(fname)
    return recData.astype(np.float32)

In [5]:
dataset = fnames_dataset.map(lambda x: tf.numpy_function(read_npy, [x], [tf.float32]))

In [6]:
it1 = iter(dataset)
ex1 = next(it1)
ex1

2022-05-06 14:16:22.467116: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-06 14:16:22.467360: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3199980000 Hz


(<tf.Tensor: shape=(2, 48069), dtype=float32, numpy=
 array([[  52.,  -17., -185., ..., -353., -381., -411.],
        [   0.,    0.,    0., ...,    4.,    4.,    4.]], dtype=float32)>,)

Poderiamos também usar a função 'map' para separar inputs e targets e obter um dataset que produz um 'tuple', (input, target) 

In [7]:
dataset2 = dataset.map(lambda x: (x[0],x[1]))
it2 = iter(dataset2)
ex2 = next(it2)
ex2

(<tf.Tensor: shape=(48069,), dtype=float32, numpy=array([  52.,  -17., -185., ..., -353., -381., -411.], dtype=float32)>,
 <tf.Tensor: shape=(48069,), dtype=float32, numpy=array([0., 0., 0., ..., 4., 4., 4.], dtype=float32)>)

O nosso dataset pode também produzir 'batches' em vez de exemplos individuais.  No nosso caso os exempos não tẽm todos o mesmo comprimento, assim, teremos de extender(por zeros??) os exemplos mais curtos de cada batch de modo a todos os exemplos do mesmo batch terem o mesmo comprimento.

In [8]:
batch_size = 2
dataset3 = dataset2.padded_batch(batch_size, padded_shapes=([None,],[None,]))
it3 = iter(dataset3)
ex3 = next(it3)
ex3

(<tf.Tensor: shape=(2, 48475), dtype=float32, numpy=
 array([[  52.,  -17., -185., ...,    0.,    0.,    0.],
        [ 115., -208., -297., ...,   15.,  281.,  314.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 48475), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 4., 4., 0.]], dtype=float32)>)

    Poderiamos também usar a função map para aplicar a 'stft' aos inputs 